In [ ]:
import json
import pandas as pd
import math
from tqdm import tqdm 
import cv2
import numpy as np
import os

import torch 
import torchvision
import torch.nn as nn 
from IPython.display import Image 
from torchvision import transforms
import matplotlib.pyplot as plt
import random
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 12345
random.seed(seed)
torch.manual_seed(seed)
import PIL
import requests
  
from sklearn import decomposition    
from sklearn.preprocessing import MinMaxScaler

# Load Data

## Real Data
create new directory with only images.

In [ ]:
real_test_role_coco_dir = "/content/drive/MyDrive/111 Rendered.ai/RarePlanes/datasets/coco_data/role_real_test_coco.json"
real_imgs_dir = "/content/drive/MyDrive/111 Rendered.ai/RarePlanes/datasets/real/test/RarePlanes_test_PS-RGB_tiled.tar.gz (Unzipped Files)/PS-RGB_tiled"
real_json_dir = real_test_role_coco_dir
with open(real_json_dir) as json_file:
    real_coco = json.load(json_file)
    
print(real_coco.keys())
print("Num images:{}".format(len(real_coco['images'])))
print("Num classes:{}".format(len(real_coco['categories'])))
print("Num bbox/instances:{}".format(len(real_coco['annotations'])))

img_list_real = pd.DataFrame(real_coco['images'])
img_list_real.head()

annotations_real = pd.DataFrame(real_coco['annotations'])
annotations_real.head()

ann_real = annotations_real.merge(img_list_real, how='left', left_on="image_id", right_on="id")
ann_real.shape

# filter
print("keep only the civil category planes, left with {} instances".format(ann_real[ann_real["category_id"] <=2].shape[0]))
civil_images_list = list(ann_real[ann_real["category_id"] <=2].file_name.unique())
print("keeping only {} images".format(len(civil_images_list)))

dict_keys(['annotations', 'categories', 'images'])
Num images:2710
Num classes:7
Num bbox/instances:6812
keep only the civil category planes, left with 6457 instances
keeping only 2601 images


In [ ]:
# cp to new images dir
from shutil import copyfile
from tqdm import tqdm

dst_dir = "/content/drive/MyDrive/111 Rendered.ai/RarePlanes/datasets/real/civil_test/"
previous_imgs_dir = "/content/drive/MyDrive/111 Rendered.ai/RarePlanes/datasets/real/test/RarePlanes_test_PS-RGB_tiled.tar.gz (Unzipped Files)/PS-RGB_tiled/"
for f in tqdm(civil_images_list):
  src = previous_imgs_dir+f
  copyfile(src, dst_dir+f)

100%|██████████| 2601/2601 [37:19<00:00,  1.16it/s]


In [ ]:
# create a new json file

# generate civil role coco file
# role_real_test_coco_path = "/content/gdrive/MyDrive/RarePlanes/datasets/real/test/role_real_coco.json" # 这里改成你的path
role_real_test_coco_path = "/content/drive/MyDrive/111 Rendered.ai/RarePlanes/datasets/coco_data/role_real_test_coco.json"
with open(role_real_test_coco_path) as json_file:
    test_coco = json.load(json_file)

civil_test_coco = {}
civil_test_coco['annotations'] = [ann for ann in test_coco['annotations'] if ann['category_id'] <= 2]
civil_test_image_ids = [ann['image_id'] for ann in test_coco['annotations'] if ann['category_id'] <= 2]
civil_test_coco['images'] = [image for image in test_coco['images'] if image['id'] in civil_test_image_ids]
civil_test_coco['categories'] = [{'id': 0, 'name': "civil_small"}, {'id': 1, 'name': "civil_medium"}, {'id': 2, 'name': "civil_large"}]

## need to save civil_test_coco
# civil_role_real_test_coco_path = "/content/gdrive/MyDrive/RarePlanes/datasets/real/civil_test/civil_role_real_coco.json" # 这里改成你的path
civil_role_real_test_coco_path = "/content/drive/MyDrive/111 Rendered.ai/RarePlanes/datasets/coco_data/civil_role_real_test_coco.json"
with open(civil_role_real_test_coco_path, 'w') as outfile:
    json.dump(civil_test_coco, outfile)